### Linear Regression with Stochastic Gradient Descent and Multiple Variables
##### Boston Housing Prices
Data is taken from Keras Datasets.

In a series of steps, we can train a simple model to update weights for our multiple parameters.  Over these updates we will minimize the mean squared error between our model's predictions and the 'real answers'.  Finally we will test our model on some testing data.


In [254]:
# Lets get our Data.

import numpy as np
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [255]:
x_train.shape

(404, 13)

##### From the documentation: `Samples contain 13 attributes of houses at different locations around the Boston suburbs in the late 1970s`

We have training data from 404 houses.  We have 13 attributes for each house 

In [256]:
y_train[0:10]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4])

This is supervised learning seeing as we have the 'correct' answers.  These must be values of the homes.

### 1) The Model:

We will use an extension of y = mx + b taught in algebra class.  We will add in mx terms for each weight m and parameter x.  I will refer to weights as theta from now on.  

So we have

\begin{equation*}
h_{ \theta}(x) = \theta_{1} * x_{1} + \theta_{2} * x_{2} + ... \theta_{j} * x_{j} + b
\end{equation*}

We will use our guesses at the weights with our x_training values to generate a prediction for the price of each house.   

### 2) Initialize Weights and Feature Normalization.

Initialize our weights for each x variable.  We can start off guessing all 0s.  
Let's also add in 1 weight for the y-intercept b, to take out one step of the model.
To do this, we will need to add a column of all 1's to our x_training values.

While we are at it, let's normalize features to make it easier for gradient descent to find local minima later on.  We will see that with features and y values on the scale of -1 to 1 that we will converge to an answer much faster.

In [257]:
thetas = np.zeros(len(x_train[0]) + 1)

def add_ones(x):
    return np.array([np.insert(arr=row, obj=0, values=1) for row in x])

def normalize_features(x):
    mean_by_col = x.mean(axis=0)
    std_dev_by_col = x.std(axis = 0)
    return (x - mean_by_col) / std_dev_by_col

def preprocess_data(x):
    normalized = normalize_features(x)
    return add_ones(normalized)
    

new_x_train = preprocess_data(x_train)
new_y_train = normalize_features(y_train)

### 3) Calculate Predictions

Based on our initial guesses of the weights, we can calculate our first predictions!  Using our model from above in conjunction with converting the intercept as a weight and some matrix math...

\begin{equation*}
h_{ \theta}(x) = \theta^T * X = \theta_{0} * x_{0} + \theta_{1} * x_{1} + \theta_{2} * x_{2} + ... \theta_{j} * x_{j}
\end{equation*}

In [258]:
def calc_prediction(thetas, x_is):
    return np.dot(x_is, thetas)

predictions = calc_prediction(thetas, new_x_train)

### 4) The cost function

We can expect our model to have a hefty cost since we just guessed all 0's for the weights.  That means we guessed all parameters to have 0 predictive power!

\begin{equation*}
Cost = \frac{1}{2m} * \sum_{i=1}^m (h_{ \theta}(x^i) - y^i)^2
\end{equation*}

In [259]:
def cost_function(ys, predictions):
    squared_error = ((predictions - ys) * y_train.std()) ** 2
    mean_squared_error = np.mean(squared_error)
    return mean_squared_error

cost = cost_function(new_y_train, predictions)

def std_dev(mse):
    return mse ** 0.5

cost

84.62225272032155

Update Rule:

We would like to minimize the cost function.  We can do this by taking the first derivative with respect to theta_j for each weight j, and adjusting our guess for theta_j by that amount times a learning rate.
 
 \begin{equation*}
 \frac{\partial}{\partial \theta_{j}}CE = \sum_{i=1}^m[ (h_{ \theta}(x^i) - y^i) * x^i_{j} ]
 \end{equation*}

As the heart of gradient descent, we will run updates across all thetas, where we subract out the derivative with respect to each individual theta.  The derivative is multiplied by a stepping rate (Learning Rate, alpha) to fine tune the steps.

\begin{equation*}
\theta_{j} = \theta_{j} - \alpha * \frac{\partial}{\partial \theta_{j}}CE = \theta_{j} - \alpha * \sum_{i=1}^m[ (h_{ \theta}(x^i) - y^i) * x^i_{j} ]
\end{equation*}


In [260]:
LEARNING_RATE = .1

def deriv_wrt_theta_j(predictions, ys, x_is, j):
    m = ys.shape[0]
    return np.sum(np.dot((predictions - ys), x_is[:,j])) / m

def update_thetas(thetas, ys, x_is):
    m = ys.shape[0]
    predictions = calc_prediction(thetas, x_is)
    new_thetas = [theta - LEARNING_RATE / m * deriv_wrt_theta_j(predictions,
                                                    ys,
                                                    x_is,
                                                    j) for j, theta in enumerate(thetas)]
    return np.array(new_thetas)



In [261]:
NUM_EPOCHS = 501
BATCH_SIZE = 16


for epoch in range(0, NUM_EPOCHS):
    for batch_start in range(0, new_x_train.shape[0], BATCH_SIZE):
        x_batch = new_x_train[batch_start: batch_start + BATCH_SIZE:,]
        y_batch = new_y_train[batch_start:batch_start + BATCH_SIZE]
        thetas = update_thetas(thetas, y_batch, x_batch)
    
    predictions = calc_prediction(thetas, new_x_train)
    cost = cost_function(new_y_train, predictions)

    if (epoch % 100 == 0):
        print(
            'EPOCH#{0}: MSE of {1:0.2f}'.format(epoch, cost)
        )
        
print(thetas[1:10])

EPOCH#0: MSE of 49.98
EPOCH#100: MSE of 22.58
EPOCH#200: MSE of 22.49
EPOCH#300: MSE of 22.49
EPOCH#400: MSE of 22.49
EPOCH#500: MSE of 22.49
[-0.12293773  0.1381039   0.00179547  0.10480701 -0.20931052  0.26430716
  0.0651462  -0.36373601  0.33347351]


We can check that our model is working properly using the normal equation

\begin{equation*}
\theta = (X^T * X)^{-1} * X^T * y
\end{equation*}

In [262]:
transpose_x = new_x_train.transpose()

x_transpose_x = np.dot(transpose_x, new_x_train)
inverse = np.linalg.inv(x_transpose_x)
m_mult_thetas = inverse.dot(transpose_x).dot(new_y_train)
m_mult_thetas[1:10]

array([-0.12039218,  0.14709038,  0.0029461 ,  0.10809323, -0.26106711,
        0.26049337,  0.02295841, -0.37734568,  0.31613628])

Great!  We are getting similar values for thetas!